# 1. Парсер vk

По каждому жанру нам нужно сделать 5 файлов для решения различных задач. Именно по этим файлам дальше будет разбита вся наша тетрадка. 

* Файл с подписчиками каждой группы жанра: 

```
    {"id группы 1" : [id подписчиков], .... }
```

* Файл с комментами каждой группы жанра: 

```
    {"id группы 1" : [комменты], .... }
```

Когда по каждому жанру будут скачаны подписчики, из низ надо сделать `set()` и по id каждого подписчика скачать: 


* Файл со стенками подписчиков: 

```
    {"подписчик 1" : [стена],  .... }
```

* Файл с топ-10 подписками по каждому подписчику: 

```
    {"подписчик 1" : [id его подписок],  .... }
```

Когда будут скачаны все паблики, из них надо сделать `set()` и по id каждого скачать: 

* Сотня последних постов с каждого паблика

```
    {"id паблика" :  [100 последних постов] , ....  }
```

In [38]:
# Загружаем необходимые библиотеки

import datetime                  # Пакет для работы с временными форматами
import pickle                    # Пакет для подгрузки данных специфического для питона формата
import requests                  # Пакет для скачки данных из этих ваших интернетов
import pandas as pd              # Пакет для работы с таблицами
import numpy as np               # Пакет для работы с векторами и матрицами
import matplotlib.pyplot as plt  # Пакет для строительства графиков
import time            # Пакет для работы со временем. Например, помогает ставить заглушки 
                       # time.sleep(секунды), необходимые для того что ВК не банил нащего
                       # сборщика данных из-за слишком частых запросов
        
# Пакет для красивых циклов. При желании его можно отключить. Тогда из всех циклов придётся 
# удалять команду tqdm_notebook.
from tqdm import tqdm_notebook   # подробнее: https://github.com/tqdm/tqdm

In [39]:
# мой номер странички
my_user_id = 91857120
# версия используемого API
version = '5.73' 
# подгружаем токен
with open('token.txt') as f:
    token = f.read()

In [40]:
def vkDownload(method, parameters, token=token, version=version):
    """
        Возвращает результат запроса по методу
        
        method: string
            метод из документации, который хотим использовать
            
        parameters: string
            параметры используемого метода
            
        token: string
            токен Oauth доступа
        
        version: string
            версия API
    """
    
    # составляем ссылку
    url = 'https://api.vk.com/method/{method}?{parameters}&access_token={token}&v={version}'
    url = url.format(method=method, parameters=parameters, token=token, version=version)
    # запрашиваем ссылку и переводим в json (словарь)
    response = requests.get(url).json()
    
    return response

vkDownload('users.get','user_ids=91857120')

{'response': [{'first_name': 'Дмитрий',
   'id': 91857120,
   'last_name': 'Сергеев'}]}

## 1.1 Подписчики каждой группы для жанра

In [41]:
music_style = 'эстрада'

target_groups ={
     'Пугачёва': 'allapugachevapro'
    }

In [42]:
# слоаврь группа-юзеры
group_users_dct = { }

Перед тем, как сохранять юзеров, надо профильтровать их на предмет существования... 

In [43]:
vkDownload('users.get','user_ids=barbizon_kid')

{'response': [{'deactivated': 'deleted',
   'first_name': 'Наталья',
   'id': 13034843,
   'last_name': 'Савостьянова'}]}

In [124]:
def checkExisting(uids):
    """
        Проверяет существование id и оставляет только не удаленные 
    """
    
    cur_inf = vkDownload('users.get','user_ids={}'.format(','.join(str(e) for e in uids)))
    cur_inf = cur_inf['response']
    clean_members = [us_inf['id'] for us_inf in cur_inf if us_inf.get('deactivated','real') == 'real']
    return clean_members


def getGroupMembers(group_id):
    """
        Возвращает список всех пользователей данной группы
        Итерации идут батчами по 1000 пользователей
        
        group_id: string
            идентификатор группы (ссылка)
    """
    
    # Узнаём число запросов, которое надо сделать 
    count = vkDownload('groups.getMembers','group_id=' + group_id)['response']['count']
    
    # выясняем, сколько запросов нам понадобиться
    n = int(np.ceil(count/1000))  
    
    # вектор, где мы будем хранить id пользователей
    members = []     
    
    for i in tqdm_notebook(range(n)): 
        # при помощи метода groups.getMembers получаем пользователей группы
        current_members = vkDownload('groups.getMembers','group_id='+group_id+'&offset='+str(1000*i))
        current_members = current_members['response']['items']
        
        # проверим реально ли существуют все юзеры из списка выше
        for i in range(0,1000,200):
            time.sleep(0.3)
            members.extend(checkExisting(current_members[i:i+200]))
        
        # перед следующим запросом немножко подождем
        time.sleep(0.4)
        
    return members

In [125]:
# по каждому id группы сохраняем подписчиков 
for k,v in tqdm_notebook(target_groups.items()):
    members = getGroupMembers(target_groups[k]) 
    print(k,':',len(members))
    group_users_dct[v] = members

HBox(children=(IntProgress(value=0, max=1), HTML(value='')))

Exception in thread Thread-19:
Traceback (most recent call last):
  File "/usr/local/Cellar/python/3.6.4_4/Frameworks/Python.framework/Versions/3.6/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/usr/local/lib/python3.6/site-packages/tqdm/_monitor.py", line 62, in run
    for instance in self.tqdm_cls._instances:
  File "/usr/local/Cellar/python/3.6.4_4/Frameworks/Python.framework/Versions/3.6/lib/python3.6/_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration



HBox(children=(IntProgress(value=0, max=33), HTML(value='')))

Пугачёва : 29140



In [126]:
group_users_dct['allapugachevapro'][-20:]

[519952826,
 519986903,
 520391736,
 520931611,
 521001719,
 521097546,
 521146892,
 521190661,
 521505972,
 521540712,
 521560189,
 521772661,
 521887515,
 521923792,
 522548600,
 522998936,
 523271589,
 523478186,
 523508174,
 523729785]

In [ ]:
# Сохраняем словарик для текущего жанра 
with open('data/{music_style}_group_users', 'wb') as f:
    pickle.dump(group_users_dct, f)

## 1.2 Файл с комментами каждой группы жанра

Скачиваем с группы все посты.

In [ ]:
# Сюда сунуть цикл, который по всем группам жанра качает id постов с офсетом вниз 

In [ ]:
def makeBatch(uids):
    """
        Возвращает лист листов из пользователей, батчами по 25 человек 
    """
    batches = [uids[i:i+25] for i in range(0,len(uids),25)]
    
    return batches 


# !!! Переписать функцию под комментарии по аналогии с лайками (см цикл ниже)

def makePublicUrl(batch, token=token):
    """
        Создаёт ссылку для метода execute на основе батчей пользователей для скачки пабликов
            batch - батч из id 
            token - ключ доступа
    """
    
    # составляем ссылку из кусочков
    begin = 'https://api.vk.com/method/execute?code=return['
    end = '];&access_token='+token+'&v=5.78'
    middle = ''
    # для каждого пользователя из батча используем метод wall.get 
    for bt in batch:
        middle += 'API.users.getSubscriptions({"user_id": "'+str(bt)+'"}),'   
    return begin + middle[:-1] + end 

# !!! Переписать под комментарии по аналогии с лайками (см цикл ниже)

def getUsersPublics(uids):
    """
    Достаёт список пабликов для всех людей из списка uids
        uids - id людей 
    """
    
    batches = makeBatch(uids) # дробим на батчи
    
    # сюда будем записывать соответсвие между юзером и пабликами 
    id_group_list = { }
    
    for item in tqdm_notebook(batches):
        time.sleep(0.3)
        response = requests.get(makePublicUrl(item))
        cur_resp = response.json()['response']
        
        # распаковываем список подписок по подписчикам
        for uid,cur in zip(item,cur_resp):
            id_group_list[uid] = cur['groups']['items'][:10] # первые 10
            
    return(id_group_list)


In [ ]:
requ = [ ]
req_info = [ ]  # здесь будет лежать информация о каждом запросе
# Проходимся по всем группам
for group_id in posts_id.keys():
    # Проходимся по всем постам в группе 
    for item in posts_id[group_id]:
        # Сдвиги в лайках для текущего поста 
        for ofset in range(item[1]//1000 + 1):
            s = 'API.likes.getList({"type":"post","owner_id":"-' + str(group_id) + \
                 '","item_id":"' + str(item[0]) + '","count":"1000","offset":"'+str(1000*ofset)+'"}),'
            s_inf = (group_id, item[0], 1000*ofset, item[1])
            requ.append(s)
            req_info.append(s_inf)

## 1.3 Файл со стенками подписчиков 

Мы скачали подписчиков по каждому из жанров. Подгрузим их сюда и сделаем по ним `set()`.

In [ ]:
# !!! Внимание! По этому методу есть лимит в 10к запросов. Имеет смысл брать жанр, 
# после делать по нему случайную выборку в 15к людей и качать стенки только по ним !!!

In [ ]:
def makeWallUrl(batch, token=token):
    """
        Создаёт ссылку для метода execute на основе батчей пользователей
    """
    # составляем ссылку из кусочков
    begin = 'https://api.vk.com/method/execute?code=return['
    end = '];&access_token='+token+'&v='+version
    middle = ''
    # для каждого пользователя из батча используем метод wall.get 
    for bt in batch:
        middle += 'API.wall.get({"owner_id": "'+str(bt)+'","count":"100"}),'
        
    return begin + middle[:-1] + end 

def getAllPosts(candidate):
    """
        Возвращает список словарей, в каждом их которых посты пользователей
    """
    
    # инициализируем список для хранения постов
    wall = []
    # берем всех людей текущего кандидата
    people = candidate_people[candidate]
    
    # рандомно перемешаем наших пользователей, чтобы обеспечить репрезентативность выборки
    np.random.shuffle(people)
    
    # создадим свой небольшой класс для кастомной ошибки 
    # на случай превышения лимита запросов
    class LimitExceeded(Exception):
        pass
    
    # батчи пользователей
    batches = makeBatch(people)
    
    # итерируем по батчам
    for item in tqdm_notebook(batches):
        # если насобирали 10 тысяч пользователей, останавливаемся
        if len(wall) < 10000:
            try:
                # пробуем выгрузить по батчу стенки пользователей
                res = requests.get(makeWallUrl(item))
                res = res.json()['response']
                
                # если нас забанили (все респонсы пусты), поднимем ошибку
                if res == [False]*25:
                    raise LimitExceeded("Превышено число запросов")
                
                # записываем полученные данные в список
                wall.extend(res)
                
            except LimitExceeded:
                # если забанили - подождем минутку, чтобы обновились лимиты запросов
                time.sleep(60)
                
            # в конце каждой итерации ждем 2 секунды, чтобы не словить бан
            time.sleep(2)
        else:
            break
            
    return wall

## 1.4 Файл с топ-10 подписками по каждому подписчику

In [23]:
unique_users = group_users_dct['allapugachevapro'][:100]
unique_users[:5]

[2022, 8088, 9954, 15067, 17110]

In [28]:
def makePublicUrl(batch, token=token):
    """
        Создаёт ссылку для метода execute на основе батчей пользователей для скачки пабликов
            batch - батч из id 
            token - ключ доступа
    """
    
    # составляем ссылку из кусочков
    begin = 'https://api.vk.com/method/execute?code=return['
    end = '];&access_token='+token+'&v=5.78'
    middle = ''
    # для каждого пользователя из батча используем метод wall.get 
    for bt in batch:
        middle += 'API.users.getSubscriptions({"user_id": "'+str(bt)+'"}),'   
    return begin + middle[:-1] + end 

def getUsersPublics(uids):
    """
    Достаёт список пабликов для всех людей из списка uids
        uids - id людей 
    """
    
    batches = makeBatch(uids) # дробим на батчи
    
    # сюда будем записывать соответсвие между юзером и пабликами 
    id_group_list = { }
    
    for item in tqdm_notebook(batches):
        time.sleep(0.3)
        response = requests.get(makePublicUrl(item))
        cur_resp = response.json()['response']
        
        # распаковываем список подписок по подписчикам
        for uid,cur in zip(item,cur_resp):
            id_group_list[uid] = cur['groups']['items'][:10] # первые 10
            
    return(id_group_list)

In [31]:
makePublicUrl(unique_users[:2])

'https://api.vk.com/method/execute?code=return[API.users.getSubscriptions({"user_id": "2022"}),API.users.getSubscriptions({"user_id": "8088"})];&access_token=a53f98e78b71d1d9fc2d4a2c90c74248a0e56e3fe9fd194f4ad5ad34b8ee17681b1b49e727952e1de8e13&v=5.78'

In [32]:
# !!! Проверить правда ли это верхние 10 пабликов или нихера !!!

getUsersPublics(unique_users[:2])

HBox(children=(IntProgress(value=0, max=1), HTML(value='')))

{2022: [49135017,
  141352710,
  107199843,
  24098496,
  33592301,
  133846520,
  39444069,
  34274053,
  50911295,
  149082322],
 8088: [29653869,
  32822100,
  31823204,
  82088213,
  166586790,
  51125160,
  28890647,
  133386541,
  93159176,
  170445225]}

In [ ]:
# Сохраняем это чудо 

## 1.5 Сотня последних постов с каждого паблика

Мы скачали все топы подписок по каждому подписчику. Теперь нужно сделать по ним `set()`.

In [ ]:
# Сделать по аналогии для 100 постов с каждого паблика !!!

def makePublicUrl(batch, token=token):
    """
        Создаёт ссылку для метода execute на основе батчей пользователей для скачки пабликов
            batch - батч из id 
            token - ключ доступа
    """
    
    # составляем ссылку из кусочков
    begin = 'https://api.vk.com/method/execute?code=return['
    end = '];&access_token='+token+'&v=5.78'
    middle = ''
    # для каждого пользователя из батча используем метод wall.get 
    for bt in batch:
        middle += 'API.users.getSubscriptions({"user_id": "'+str(bt)+'"}),'   
    return begin + middle[:-1] + end 

def getUsersPublics(uids):
    """
    Достаёт список пабликов для всех людей из списка uids
        uids - id людей 
    """
    
    batches = makeBatch(uids) # дробим на батчи
    
    # сюда будем записывать соответсвие между юзером и пабликами 
    id_group_list = { }
    
    for item in tqdm_notebook(batches):
        time.sleep(0.3)
        response = requests.get(makePublicUrl(item))
        cur_resp = response.json()['response']
        
        # распаковываем список подписок по подписчикам
        for uid,cur in zip(item,cur_resp):
            id_group_list[uid] = cur['groups']['items'][:10] # первые 10
            
    return(id_group_list)